# Raw spectrum value check

> Check that all raw spectra have real numbers and unsaturated pixels in their counts arrays.

In [ ]:
# |default_exp diagnostics.raw_spectrum_value_check

In [ ]:
# |export

import numpy as np

from qagmire.data import (
    get_lr_raw_files,
    read_raw_data,
)
from qagmire.quality_assurance import Diagnostics

To write checks of the data, we create a subclass of `Diagnostics` and implement the `tests` method.

In [ ]:
# |export


class RawSpectrumValueCheck(Diagnostics):
    """Raw spectrum value check.

    A reproduction of the class with the same name in the weaveio
    [value_checks](https://github.com/bamford/QAG/blob/master/diagnostics/value_checks.py).

    This tests for the following cases:

    * Are there too many pixels saturated above the ADU threshold?
    * Are there negative pixel values?
    * Are there non-finite pixel values?

    for both counts1 and counts2.
    """

    def __init__(
        self,
        saturation_limit_adu: float = 65535,  # the saturation limit in ADU
        n_allowed_saturated_pixels: int = 0,  # the number of allowed saturated pixels per counts array
        **kwargs,  # additional keyword arguments are passed to the `Diagnostics` constructor
    ):
        self.saturation_limit_adu = saturation_limit_adu
        self.n_allowed_saturated_pixels = n_allowed_saturated_pixels
        super().__init__(**kwargs)

    def tests(self, **kwargs):
        files = get_lr_raw_files(**kwargs)
        data = read_raw_data(files)

        sat = data >= self.saturation_limit_adu
        neg = data < 0
        nan = ~np.isfinite(data)

        count_sat = sat.sum(dim=["dim_0", "dim_1"])
        any_neg = neg.any(dim=["dim_0", "dim_1"])
        any_nan = nan.any(dim=["dim_0", "dim_1"])

        tests = [
            {
                "name": "too_many_sat_in_counts1",
                "description": "Are there too many pixels saturated above the ADU threshold in counts1?",
                "test": count_sat["counts1"] > self.n_allowed_saturated_pixels,
            },
            {
                "name": "neg_pixels_in_counts1",
                "description": "Are there negative pixel values in counts1?",
                "test": any_neg["counts1"],
            },
            {
                "name": "nan_pixels_in_counts1",
                "description": "Are there non-finite pixel values in counts1?",
                "test": any_nan["counts1"],
            },
            {
                "name": "too_many_sat_in_counts2",
                "description": "Are there too many pixels saturated above the ADU threshold in counts2",
                "test": count_sat["counts2"] > self.n_allowed_saturated_pixels,
            },
            {
                "name": "neg_pixels_in_counts2",
                "description": "Are there negative pixel values in counts2?",
                "test": any_neg["counts2"],
            },
            {
                "name": "nan_pixels_in_counts2",
                "description": "Are there non-finite pixel values in counts2?",
                "test": any_nan["counts2"],
            },
        ]
        return tests

## Demonstration tests

Here we use multiple `dask` workers to speed up this test. We are checking 10 billion pixel values. On a single core, this takes ~110 seconds. With 8 workers, it takes ~22 seconds.

In [ ]:
tests = RawSpectrumValueCheck(n_processes=8)
tests.run(date="201*")

Locating and converting where necessary:   0%|                | 0/252 [00:00<?, ?it/s]

Locating and converting where necessary:   0%|        | 1/252 [00:01<07:00,  1.68s/it]

Locating and converting where necessary:   2%|▏       | 5/252 [00:01<01:08,  3.63it/s]

Locating and converting where necessary:   3%|▏       | 7/252 [00:01<00:50,  4.87it/s]

Locating and converting where necessary:   4%|▎       | 9/252 [00:03<01:29,  2.71it/s]

...

Locating and converting where necessary: 100%|██████| 252/252 [00:38<00:00,  6.55it/s]


Reading netCDF files... 

took 4.58 s. Size is 37566.851 Mb
Tests took 50.49 s to prepare (including reading data).


2024-03-19 16:58:42,116 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/utils_comm.py", line 455, in retry_operation
    return await retry(
           ^^^^^^^^^^^^
  File "/home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-pack

Tests took 27.21 s to perform.
too_many_sat_in_counts1:
    Are there too many pixels saturated above the ADU threshold in counts1?
neg_pixels_in_counts1:
    Are there negative pixel values in counts1?
nan_pixels_in_counts1:
    Are there non-finite pixel values in counts1?
too_many_sat_in_counts2:
    Are there too many pixels saturated above the ADU threshold in counts2
neg_pixels_in_counts2:
    Are there negative pixel values in counts2?
nan_pixels_in_counts2:
    Are there non-finite pixel values in counts2?


In [ ]:
tests.summary()

failed  \
test                                             too_many_sat_in_counts1   
filename RUN     CAMERA MJD        NIGHT    OBID                           
r1002213 1002213 RED    57639.8653 20160908 3191                    True   
r1002215 1002215 RED    57639.8784 20160908 3191                    True   
r1002217 1002217 RED    57639.8916 20160908 3191                    True   
r1002219 1002219 RED    57639.9048 20160908 3191                   False   
r1002243 1002243 RED    57639.9940 20160908 3346                   False   
r1002307 1002307 RED    57640.9996 20160909 3217                   False   
r1003335 1003335 RED    57809.0648 20170224 3900                   False   

                                                                          \
test                                             too_many_sat_in_counts2   
filename RUN     CAMERA MJD        NIGHT    OBID                           
r1002213 1002213 RED    57639.8653 20160908 3191                   False   
r1002215 1002215 RED    57639.8784 20160908 3191                   False   
r1002217 1002217 RED    57639.8916 20160908 3191                   False   
r1002219 1002219 RED    57639.9048 20160908 3191                    True   
r1002243 1002243 RED    57639.9940 20160908 3346                    True   
r1002307 1002307 RED    57640.9996 20160909 3217                    True   
r1003335 1003335 RED    57809.0648 20170224 3900                    True   

                                                 total fails  
test                                                          
filename RUN     CAMERA MJD        NIGHT    OBID              
r1002213 1002213 RED    57639.8653 20160908 3191           1  
r1002215 1002215 RED    57639.8784 20160908 3191           1  
r1002217 1002217 RED    57639.8916 20160908 3191           1  
r1002219 1002219 RED    57639.9048 20160908 3191           1  
r1002243 1002243 RED    57639.9940 20160908 3346           1  
r1002307 1002307 RED    57640.9996 20160909 3217           1  
r1003335 1003335 RED    57809.0648 20170224 3900           1

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()